In [14]:
import sys
sys.path.append("/scratch/gpfs/graceliu/contrastive_rl")

import copy
import functools

from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

import jax
import optax
import numpy as np
from acme import specs
import tensorflow as tf

from acme.tf.savers import SaveableAdapter

from contrastive.config import ContrastiveConfig
from contrastive import utils as contrastive_utils
from contrastive import make_networks
from contrastive.utils import make_environment
from contrastive import ContrastiveLearner
from acme.utils import counting

# disable tensorflow_probability warning: The use of `check_types` is deprecated and does not have any effect.
import logging
logger = logging.getLogger("root")

class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()

logger.addFilter(CheckTypesFilter())

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import seaborn as sn
import pandas as pd
import pickle
from scipy import stats
import imageio

In [15]:
def load_checkpoint(alpha, env_name, log_dir, seed, fix_goals = False, ckpt_num = None):
    state_entropy_coefficient = alpha
    if(alg == 'contrastive_cpc'):
        misc_params = '{}_None'.format(alpha)
    else:
        misc_params = '{}_0.0'.format(alpha)
    
    ckpt_dir = '{}/{}_{}_{}_{}/checkpoints/learner'.format(log_dir, alg, env_name, misc_params, seed)
    ckpt_counter_dir = '{}/{}_{}_{}_{}/checkpoints/counter'.format(log_dir, alg, env_name, misc_params, seed)
    
    fixed_start_end = None
    fixed_goal = False
        
    if env_name=='sawyer_box':
        # this value doesn't actually matter for the sawyer box and peg environment since the goal position is overwritten in env_utils.py
        fixed_start_end = np.array([0.0, 0.75, 0.133])
    elif env_name=='sawyer_bin':
        fixed_start_end = np.array([0.12, 0.7, 0.02])
    elif env_name=='sawyer_peg':
        fixed_start_end = np.array([-0.3, 0.6, 0.0])
    else:
        fixed_start_end = None
    
    
    env_factory = lambda seed: make_environment(env_name, config.start_index, 
                                                config.end_index, seed=np.random.randint(1e6), fixed_start_end=fixed_start_end)[0] 
    dummy_seed = 1
    environment_spec = specs.make_environment_spec(env_factory(np.random.randint(1e6)))

    obs_dim = make_environment(env_name, config.start_index, config.end_index, seed=np.random.randint(1e6), fixed_start_end=fixed_start_end)[1]
                                   
    network_factory = functools.partial(
      make_networks, obs_dim=obs_dim, repr_dim=config.repr_dim,
      repr_norm=config.repr_norm, twin_q=False,
      use_image_obs=config.use_image_obs,
      hidden_layer_sizes=config.hidden_layer_sizes,
    )

    random_key = jax.random.PRNGKey(np.random.choice(int(1e6)))
    networks = network_factory(environment_spec)
    policy_optimizer = optax.adam(
      learning_rate=config.actor_learning_rate)
    q_optimizer = optax.adam(.001)#learning_rate=config.critic_learning_rate

    trained_learner = ContrastiveLearner(
      networks=networks,
      rng=random_key,
      policy_optimizer=policy_optimizer,
      q_optimizer=q_optimizer,
      iterator=None,
      counter=None,
      logger=None,
      obs_to_goal=functools.partial(contrastive_utils.obs_to_goal_2d,
                                    start_index=config.start_index,
                                    end_index=config.end_index),
      config=config)

    returns_list = []
    success_rate_list = []
    
    env = env_factory(np.random.randint(1e6))
    obs_dim = env.observation_spec().shape[0] // 2
    episode_returns = np.zeros([NUM_EPISODES, ])

    ckpt = tf.train.Checkpoint(learner=SaveableAdapter(trained_learner))
    ckpt_mgr = tf.train.CheckpointManager(ckpt, ckpt_dir, 1)
    
    counter = counting.Counter()
    ckpt_counter = tf.train.Checkpoint(counter=SaveableAdapter(counter))
    ckpt_mgr_counter = tf.train.CheckpointManager(ckpt_counter, ckpt_counter_dir, 1)
    if ckpt_num is not None:
        ckpt.restore(ckpt_dir + '/ckpt-' + str(ckpt_num)).assert_consumed()
        ckpt_counter.restore(ckpt_counter_dir + '/ckpt-' + str(ckpt_num)).assert_consumed()
        print(ckpt_dir + str(ckpt_num))
    else:
        ckpt.restore(ckpt_mgr.latest_checkpoint).assert_consumed()
        ckpt_counter.restore(ckpt_counter_dir + '/ckpt-' + str(ckpt_num)).assert_consumed()
        print(ckpt_mgr.latest_checkpoint)

    trained_learner_state = trained_learner._state
    counts = counter.get_counts()
    print('actor steps: ' + str(counts['actor_steps']))
    print("Model loaded from: {}".format(ckpt_dir))
    return trained_learner_state, env, networks

In [16]:
def gen_images(alpha, env_name, log_dir, seed, ckpt_list, num_epi_per_ckpt = 1):    
    
    imgs_list = []
    
    for ckpt_num in ckpt_list:
        trained_learner_state, env, networks = load_checkpoint(alpha, env_name, log_dir, seed, fix_goals=True, ckpt_num = ckpt_num)
        episode_returns = np.zeros([num_epi_per_ckpt, ])
        
        for epi in range(num_epi_per_ckpt):
          t = 0
          env.seed(epi)  # use fixed seed for different methods
          timestep = env.reset()
          episode_return = 0

          while not timestep.last():
            # render images
            #img = env.render(offscreen=True, camera_name="corner3", resolution=(320, 240)) 
            img = env.render(offscreen=True, camera_name="corner3", resolution=(1280, 960))
            imgs_list.append(img)

            dist = networks.policy_network.apply(
              trained_learner_state.policy_params,
              timestep.observation
            )
            action = np.array(dist.mode())
            timestep = env.step(action)

            # Book-keeping.
            t += 1
            episode_return += timestep.reward

          # assert t == env._step_limit
          print("episode return = {}".format(episode_return))
          episode_returns[epi] = episode_return
        
    return imgs_list

In [17]:
def play_video(imgs_list, save_video=False):
    for imgs in imgs_list:
        fig = plt.figure()
        plt.axis('off')
        plt.tick_params(axis='both', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)
        im = plt.imshow(imgs[0, :, :, :])
        

        plt.close() # this is required to not display the generated image

        def init():
            im.set_data(imgs[0, :, :, :])

        def animate(i):
            im.set_data(imgs[i, :, :, :])
            return im

        anim = animation.FuncAnimation(fig, animate, init_func=init, frames=imgs.shape[0],
                                       interval=100)
        display(HTML(anim.to_html5_video()))
        if save_video:
            writervideo = animation.FFMpegWriter(fps=60) 
            anim.save('final_paper_exp/videos/test.mp4', writer=writervideo) 

In [18]:
def save_video(imgs_list, file_path):
    imageio.mimwrite(file_path, imgs_list)

In [19]:
def play_video(imgs_list, save_video=False):
    fig = plt.figure()
    plt.axis('off')
    plt.tick_params(axis='both', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)

    anim = animation.ArtistAnimation(fig, imgs_list, interval=50, blit=True,
                                repeat_delay=1000)
    display(HTML(anim.to_html5_video()))
    if save_video:
        writervideo = animation.FFMpegWriter(fps=60) 
        anim.save('final_paper_exp/videos/test.mp4', writer=writervideo) 
    plt.close()

In [20]:
config = ContrastiveConfig()
seeds_list = [0, 1, 2, 3, 4]
IMAGE_HEIGHT = 960
IMAGE_WIDTH = 1280
NUM_EPISODES = 1000

config.use_cpc_symm = True
config.use_goal_action = True
config.use_quasimetric_logit = True
config.use_goal_potential = True
config.twin_q = True
fixed_goal=False
goal_difficulty = None
fixed_goal_coords=[0.12, 0.7, 0.02]
goal_difficulty_list = [0.2, 0.6, 1.0]
fix_point_goals = True

use_l2_critic = False
use_phi_critic = True
use_sa_reg = True
is_kl = False

NUM_EPISODES = 1
fix_point_goals = True
env_name = 'sawyer_box'
EPISODE_LENGTH = 150

obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4
action_list = [[1,0], [1,1], [0,1], [0,0], [-1,0], [-1,-1], [0,-1], [-1,1], [1,-1]]
observation = np.array([5,5,10,10], dtype=float)

# Fixed goal - Saywer Box

In [8]:
alpha = "0.1"
env_name = 'sawyer_box'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 1
ckpt_list = list(range(2, 22))

In [9]:
obs_repr_shape = 22
state_repr_shape = 11
action_repr_shape = 4

In [10]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.interpreters.xla.DeviceArray is deprecated. Use jax.Array instead.
  warnings.warn(message, DeprecationWarning)
2024-07-23 19:05:52.902609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/mujoco_py/binaries/linux/mujoco210/bin
2024-07-23 19:05:52.902701: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner2
actor steps: 478350
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner3
actor steps: 971400
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner4
actor steps: 1470300
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner5
actor steps: 1968000
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bo

In [11]:
file_path = 'final_paper_exp/videos/sawyer_box_fixed.mp4'
save_video(imgs_list, file_path)

In [12]:
alpha = "0.1"
env_name = 'sawyer_box'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 3
ckpt_list = list(range(2, 22))

In [13]:
obs_repr_shape = 22
state_repr_shape = 11
action_repr_shape = 4

In [14]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner2
actor steps: 496800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner3
actor steps: 1004550
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner4
actor steps: 1513650
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_box_0.1_None_3/checkpoints/learner5
actor steps: 2020800
Model loaded from: /scratch/gpfs/graceliu/crl_final

In [15]:
file_path = 'final_paper_exp/videos/sawyer_box_random.mp4'
save_video(imgs_list, file_path)

In [16]:
alpha = "0.1"
env_name = 'sawyer_bin'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 3
ckpt_list = list(range(2, 22))

In [17]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [18]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner2
actor steps: 446250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner3
actor steps: 903450
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner4
actor steps: 1364550
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner5
actor steps: 1822350
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bi

In [19]:
file_path = 'final_paper_exp/videos/sawyer_bin_fixed.mp4'
save_video(imgs_list, file_path)

In [20]:
alpha = "0.1"
env_name = 'sawyer_bin'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 3
ckpt_list = list(range(2, 22))

In [21]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [22]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner2
actor steps: 489900
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner3
actor steps: 986250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner4
actor steps: 1481850
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 14.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner5
actor steps: 1974900
Model loaded from: /scratch/gpfs/graceliu/crl_final

In [23]:
file_path = 'final_paper_exp/videos/sawyer_bin_random.mp4'
save_video(imgs_list, file_path)

In [11]:
alpha = "0.1"
env_name = 'sawyer_peg'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [12]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [13]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner2
actor steps: 481800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner3
actor steps: 974250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner4
actor steps: 1465800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner5
actor steps: 1955850
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_pe

In [14]:
file_path = 'final_paper_exp/videos/sawyer_peg_fixed.mp4'
save_video(imgs_list, file_path)

In [15]:
alpha = "0.1"
env_name = 'sawyer_peg'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [16]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [17]:
imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)

/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner2
actor steps: 477750
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner3
actor steps: 964500
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner4
actor steps: 1449900
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval/contrastive_cpc_sawyer_peg_0.1_None_0/checkpoints/learner5
actor steps: 1934100
Model loaded from: /scratch/gpfs/graceliu/crl_final/

In [18]:
file_path = 'final_paper_exp/videos/sawyer_peg_random.mp4'
save_video(imgs_list, file_path)

In [8]:
alpha = "0.1"
env_name = 'sawyer_box'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [9]:
obs_repr_shape = 22
state_repr_shape = 11
action_repr_shape = 4

In [10]:
seed_list = [1, 2, 3, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_box_fixed_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)

/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.interpreters.xla.DeviceArray is deprecated. Use jax.Array instead.
  warnings.warn(message, DeprecationWarning)
2024-07-26 11:14:08.407125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/mujoco_py/binaries/linux/mujoco210/bin
2024-07-26 11:14:08.407196: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner2
actor steps: 478350
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner3
actor steps: 971400
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner4
actor steps: 1470300
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_1/checkpoints/learner5
actor steps: 1968000
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bo

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner13
actor steps: 5570700
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner
episode return = 47.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner14
actor steps: 6035250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner
episode return = 78.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner15
actor steps: 6500250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner
episode return = 7.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_2/checkpoints/learner16
actor steps: 6967500
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_s

episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner4
actor steps: 1125750
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner5
actor steps: 1480050
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner6
actor steps: 1886700
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_box_0.1_None_4/checkpoints/learner7
actor steps: 2316900
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/co

In [ ]:
alpha = "0.1"
env_name = 'sawyer_box'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [ ]:
obs_repr_shape = 22
state_repr_shape = 11
action_repr_shape = 4

In [ ]:
seed_list = [1, 2, 3, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_box_random_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)

In [22]:
alpha = "0.1"
env_name = 'sawyer_bin'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [9]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [10]:
seed_list = [3, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_bin_fixed_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)

/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.interpreters.xla.DeviceArray is deprecated. Use jax.Array instead.
  warnings.warn(message, DeprecationWarning)
2024-07-26 13:07:05.253155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/graceliu/.conda/envs/ris/lib/python3.9/site-packages/mujoco_py/binaries/linux/mujoco210/bin
2024-07-26 13:07:05.253224: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner2
actor steps: 446250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner3
actor steps: 903450
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner4
actor steps: 1364550
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_3/checkpoints/learner5
actor steps: 1822350
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bi

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner13
actor steps: 5820600
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 61.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner14
actor steps: 6300000
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 32.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner15
actor steps: 6778800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 61.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner16
actor steps: 7257600
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_

In [23]:
seed_list = [4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_bin_fixed_4_lift.mp4'
    save_video(imgs_list, file_path)

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner2
actor steps: 485250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner3
actor steps: 974250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner4
actor steps: 1462800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner5
actor steps: 1950150
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bi

In [28]:
seed_list = [4]
ckpt_list = [3,4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_bin_fixed_4_lift2.mp4'
    save_video(imgs_list, file_path)

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner3
actor steps: 974250
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner4
actor steps: 1462800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_bin_0.1_None_4/checkpoints/learner
episode return = 0.0


In [ ]:
alpha = "0.1"
env_name = 'sawyer_bin'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [ ]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [ ]:
seed_list = [1, 2, 3, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_bin_random_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)

In [11]:
alpha = "0.1"
env_name = 'sawyer_peg'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [12]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [13]:
seed_list = [1, 2, 3, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_peg_fixed_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner2
actor steps: 491700
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner3
actor steps: 993300
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner4
actor steps: 1493550
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_1/checkpoints/learner5
actor steps: 1992750
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_pe

/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner13
actor steps: 5931750
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner
episode return = 101.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner14
actor steps: 6422850
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner15
actor steps: 6912600
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner
episode return = 97.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_2/checkpoints/learner16
actor steps: 7402650
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_

episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner4
actor steps: 1501350
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner5
actor steps: 2002800
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner6
actor steps: 2504550
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner
episode return = 0.0
/scratch/gpfs/graceliu/crl_final/cpc/fixed/contrastive_cpc_sawyer_peg_0.1_None_4/checkpoints/learner7
actor steps: 3006150
Model loaded from: /scratch/gpfs/graceliu/crl_final/cpc/fixed/co

In [ ]:
alpha = "0.1"
env_name = 'sawyer_peg'
log_dir = '/scratch/gpfs/graceliu/crl_final/cpc/fixed_eval'
alg = 'contrastive_cpc'
seed = 0
ckpt_list = list(range(2, 22))

In [ ]:
obs_repr_shape = 14
state_repr_shape = 7
action_repr_shape = 4

In [ ]:
seed_list = [1, 2, 4]
for seed in seed_list:
    imgs_list = gen_images(alpha, env_name, log_dir, seed, ckpt_list)
    file_path = 'final_paper_exp/videos/sawyer_peg_random_' + str(seed) + '.mp4'
    save_video(imgs_list, file_path)